In [245]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import average_precision_score


In [129]:
scored_train_data = pd.read_csv('preped_train_data_wth_dummies.csv', index_col=0)
all_sales = pd.read_csv('pp_sales_jan23_avr23.csv', index_col=0)


In [239]:
user_id = 'zzWUNSYQ2CWwopiILtUU4kaBEm13'
user_information = scored_train_data[['User Key', 'Recency', 'Frequency', 'Monetary']]
available_dates = ['2023-03-26', '2023-04-18', '2023-04-08']
available_sales = all_sales[all_sales['Start At'].isin(available_dates)]
model = pickle.load(open('reg_model.pkl', 'rb'))
training_columns = ['log_Followers', 'log_Brand Appearance', 'log_Avg Price',
       'Avg Discount', 'log_First Day Revenue', 'Conversion', 'Recency',
       'Frequency', 'Monetary', 'Artisanal', 'Bien-être', 'Bio',
       'Biodégradable', 'Cadeau idéal', 'Concept original', 'Durable',
       'Exclusivité Choose', 'Fabrication à la demande', 'Fait main',
       'Gluten Free', 'Iconique', 'Inclusive', 'Innovation',
       'La Fournée Ulule', 'Made in Europe', 'Made in France', 'Naturel',
       'Oeko-Tex', 'Premium', 'Recyclable', 'Savoir-faire',
       'Socialement engagée', 'Série limitée', 'Tendance', 'Upcycling',
       'Vegan', 'ZéroDéchet', 'Éco-friendly', 'Accessoires', 'Beauté',
       'Bibliothèque', 'Bien-Être', 'Bijoux', 'Buanderie', 'Chambre',
       'Chaussures', 'Cuisine', 'Cures', 'Expériences', 'Hygiène', 'Lingerie',
       'Maroquinerie', 'Outdoor', 'Prêt-à-porter', 'Salon', 'Soins',
       'Sportswear']

In [102]:
def creating_entry_data(user_id, user_information, available_sales: pd.DataFrame):
    """ 
    Aims at returning ranked sales for a given user
    """

    user_specific_information = user_information[user_information['User Key'] == user_id].drop('User Key', axis = 1)


    user_features = user_specific_information.columns

    df_for_predictions = available_sales.__deepcopy__().reset_index(drop = True)
    user_specific_df = pd.concat([user_specific_information] *len (df_for_predictions), ignore_index=True)

    df_for_predictions[user_features] = user_specific_df

    return df_for_predictions


In [204]:
def ranking(df_for_prediction: pd.DataFrame, model, prediction_cols):

    predicted = pd.DataFrame(df_for_prediction['Sale ID'])
    

    # df_for_prediction = df_for_prediction.drop(columns=['Sale ID', 'Start At'])

    
    predicted_score = model.predict(df_for_prediction[training_columns])
    predicted['score'] = [i for i in predicted_score.flatten()]

    ordered_sales = predicted.sort_values('score')

    return ordered_sales

In [199]:
df_for_pred = creating_entry_data('yQHcG2KlpVg6TKKY0IOMI1JqDqo1',user_information, available_sales)

In [242]:
predict = ranking(df_for_pred, model, training_columns)

In [240]:
user_specific = scored_train_data[(scored_train_data['User Key'] == user_id ) & (scored_train_data['interaction'] ==  1)]

truth = pd.DataFrame(available_sales['Sale ID'])
truth['score'] = available_sales['Sale ID'].isin(user_specific['Sale ID']).astype(int)
